## Data access:

BII data: This dataset is part of the MPC STAC catalog. You will need to access the ‘io-biodiversity’ collection and look for the 2017 and 2020 rasters covering Phoenix subdivision. You can use the following coordinates for a bounding box: 
- [-112.826843, 32.974108, -111.184387, 33.863574]


Phoenix subdivision: You will find the Phoenix subdivision polygon in the Census County Subdivision shapefiles for Arizona: https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2022&layergroup=County+Subdivisions


In [16]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt
import os 

from shapely.geometry import Polygon

# used to access STAC catalogs
from pystac_client import Client
# used to sign items from the MPC STAC catalog
import planetary_computer

# ----- other libraries for nice ouputs
from IPython.display import Image

In [2]:
# access catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [3]:
# get collections and print their names
collections = list(catalog.get_collections()) # converts to a list in order to override the lazy

print('Number of collections:', len(collections))
print("Collections IDs:")
for collection in collections:
    print('- ', collection.id)

Number of collections: 122
Collections IDs:
-  daymet-annual-pr
-  daymet-daily-hi
-  3dep-seamless
-  3dep-lidar-dsm
-  fia
-  sentinel-1-rtc
-  gridmet
-  daymet-annual-na
-  daymet-monthly-na
-  daymet-annual-hi
-  daymet-monthly-hi
-  daymet-monthly-pr
-  gnatsgo-tables
-  hgb
-  cop-dem-glo-30
-  cop-dem-glo-90
-  goes-cmi
-  terraclimate
-  nasa-nex-gddp-cmip6
-  gpm-imerg-hhr
-  gnatsgo-rasters
-  3dep-lidar-hag
-  3dep-lidar-intensity
-  3dep-lidar-pointsourceid
-  mtbs
-  noaa-c-cap
-  3dep-lidar-copc
-  modis-64A1-061
-  alos-fnf-mosaic
-  3dep-lidar-returns
-  mobi
-  landsat-c2-l2
-  era5-pds
-  chloris-biomass
-  kaza-hydroforecast
-  planet-nicfi-analytic
-  modis-17A2H-061
-  modis-11A2-061
-  daymet-daily-pr
-  3dep-lidar-dtm-native
-  3dep-lidar-classification
-  3dep-lidar-dtm
-  gap
-  modis-17A2HGF-061
-  planet-nicfi-visual
-  gbif
-  modis-17A3HGF-061
-  modis-09A1-061
-  alos-dem
-  alos-palsar-mosaic
-  deltares-water-availability
-  modis-16A3GF-061
-  modis-21

In [4]:
io_biodiversity_collection = catalog.get_child('io-biodiversity')
io_biodiversity_collection

<CollectionClient id=io-biodiversity>

In [5]:
# Temporal range of interest
time_range = "2017-01-01/2020-01-01"

# Mariposa subdivision bounding box (as a GeoJSON)
bbox = {
    "type": "Polygon",
    "coordinates":[
        [
            [-112.826843, 32.974108],
            [-112.826843, 33.863574],
            [-111.184387, 33.863574],
            [-111.184387, 32.974108],
            [-112.826843, 32.974108]
        ]
    ],
}


# catalog search
search = catalog.search(
    collections=['io-biodiversity'],
    intersects=bbox,
    datetime=time_range)
search

In [6]:
items = search.item_collection()
len(items)

4

In [7]:
items

In [8]:
bii_2020 = items[0]

bii_2017 = items[3]

In [9]:
# print item id and properties
print('id:' , bii_2017.id)
bii_2017.properties

id: bii_2017_34.74464974521749_-115.38597824385106_cog


{'datetime': None,
 'proj:epsg': 4326,
 'proj:shape': [7992, 7992],
 'end_datetime': '2017-12-31T23:59:59Z',
 'proj:transform': [0.0008983152841195215,
  0.0,
  -115.38597824385106,
  0.0,
  -0.0008983152841195215,
  34.74464974521749,
  0.0,
  0.0,
  1.0],
 'start_datetime': '2017-01-01T00:00:00Z'}

In [10]:
for key in bii_2017.assets.keys():
    print(key, '--', bii_2017.assets[key].title)

data -- Biodiversity Intactness
tilejson -- TileJSON with default rendering
rendered_preview -- Rendered preview


In [12]:
bii_2017_data = rioxr.open_rasterio(bii_2017.assets['data'].href)
bii_2017_data

<xarray.DataArray (band: 1, y: 7992, x: 7992)>
[63872064 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -115.4 -115.4 -115.4 ... -108.2 -108.2 -108.2
  * y            (y) float64 34.74 34.74 34.74 34.74 ... 27.57 27.57 27.57 27.57
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [13]:
bii_2020_data = rioxr.open_rasterio(bii_2020.assets['data'].href)
bii_2020_data

<xarray.DataArray (band: 1, y: 7992, x: 7992)>
[63872064 values with dtype=float32]
Coordinates:
  * band         (band) int64 1
  * x            (x) float64 -115.4 -115.4 -115.4 ... -108.2 -108.2 -108.2
  * y            (y) float64 34.74 34.74 34.74 34.74 ... 27.57 27.57 27.57 27.57
    spatial_ref  int64 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0

In [33]:
subdivisions = gpd.read_file(os.path.join(os.getcwd(), 'data', 'tl_2022_04_cousub','tl_2022_04_cousub.shp'))
#subdivisions
subdivisions[subdivisions.NAME=='Maricopa']

,STATEFP,COUNTYFP,COUSUBFP,COUSUBNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,CNECTAFP,NECTAFP,NCTADVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry


Create a map showing the Phoenix subdivision within an appropriate geographical context. You may use any vector datasets to create your map. (You can also check out the contextily package.)

Calculate the percentage of area of the Phoenix subdivision with a BII of at least 0.75 in 2017. Obtain the same calculation for 2020.

HINTS (useful or not depending on your workflow): 
    
- Let x be an xarray.DataArray. We can select all the values greater than n by simply doing x>n. This will return an xarray.DataArray with boolean values. You can then transform this into an xarray.DataArray with 0s and 1s (instead of True/False) by casting it as type ‘int’. 
- To calculate the percentage area: (pixels in class)/(total pixels) * 100. 



Create a visualization showing the area with BII>=0.75 in 2017 that was lost by 2020. Here’s an example:
